In [22]:
import pandas as pd
players_df = pd.read_excel(r"D:\FPL\GW1\fpl_players_25_26_GW1Stats.xlsx")

In [ ]:
import pandas as pd

# Load Excel files
players_df = pd.read_excel("fpl_players_24_25.xlsx")
fixtures_df = pd.read_excel("fpl_fixtures_named_24_25.xlsx")

# Preview available position labels in the dataset
print("Available positions in player data:", players_df["position"].unique())

# Compute average team difficulty for next 5 GWs
difficulty_dict = {}

for team in players_df['team_name'].unique():
    print(team)
    team_fixtures = fixtures_df[
        (fixtures_df['home_team'] == team) | (fixtures_df['away_team'] == team)
    ].copy()

    team_fixtures = team_fixtures.sort_values(by='gameweek').head(5)
    print(team_fixtures)

    total_diff = 0
    for _, row in team_fixtures.iterrows():
        if row['home_team'] == team:
            total_diff += row['home_team_difficulty']
        else:
            total_diff += row['away_team_difficulty']

    avg_difficulty = total_diff / 5 if not team_fixtures.empty else 3
    difficulty_dict[team] = avg_difficulty

# Map and transform difficulty
players_df["avg_team_difficulty"] = players_df["team_name"].map(difficulty_dict)
players_df["difficulty_score"] = (6 - players_df["avg_team_difficulty"])

# Print difficulty scores for verification
print("\n=== Team Difficulty Scores ===")
print(players_df[["team_name", "avg_team_difficulty", "difficulty_score"]].drop_duplicates().sort_values("difficulty_score", ascending=False))


Available positions in player data: ['MID' 'FWD' 'GKP' 'DEF']
Liverpool
    gameweek  home_team    away_team  home_team_difficulty  \
0          1  Liverpool  Bournemouth                     3   
19         2  Newcastle    Liverpool                     4   
29         3  Liverpool      Arsenal                     4   
38         4    Burnley    Liverpool                     4   
40         5  Liverpool      Everton                     2   

    away_team_difficulty  
0                      5  
19                     4  
29                     5  
38                     2  
40                     5  
Man Utd
    gameweek home_team away_team  home_team_difficulty  away_team_difficulty
8          1   Man Utd   Arsenal                     4                     3
18         2    Fulham   Man Utd                     3                     3
22         3   Man Utd   Burnley                     2                     3
39         4  Man City   Man Utd                     3                     4


In [37]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load Excel files
players_df = pd.read_excel(r"D:\FPL\GW1\fpl_players_merged_with_GW1.xlsx")
fixtures_df = pd.read_excel("fpl_fixtures_named_24_25.xlsx")

# Average team difficulty for GW 1–5
first_5_gws = fixtures_df[fixtures_df['gameweek'] <= 5]
home = first_5_gws.groupby("home_team")["home_team_difficulty"].mean()
away = first_5_gws.groupby("away_team")["away_team_difficulty"].mean()
avg_difficulty = pd.concat([home, away], axis=1).mean(axis=1).reset_index()
avg_difficulty.columns = ['team_name', 'avg_team_difficulty']



# Merge difficulty into players data - FIXED: changed left_on from "team" to "team_name"
players_df = players_df.merge(avg_difficulty, left_on="team_name", right_on="team_name", how="left")

# Filter only available players
players_df = players_df[players_df["status"] == "a"]

# Normalize relevant numeric columns
norm_cols = [
    'ep_next', 'points_per_game', 'selected_by_percent', 'ict_index',
    'bps', 'bonus', 'yellow_cards', 'red_cards', 'avg_team_difficulty'
]
scaler = MinMaxScaler()
players_df[[col + "_norm" for col in norm_cols]] = scaler.fit_transform(players_df[norm_cols])

# Convert penalties_order to scaled score
def process_penalty_order(val):
    if val == 1:
        return 1.0
    elif val == 2:
        return 0.5
    else:
        return 0.0

players_df['penalties_order'] = players_df['penalties_order'].apply(process_penalty_order)


# Define position-based weights (summing to 10) - ADDED penalties_order to MID and FWD
WEIGHTS = {
    "GKP": {
        'total_points': 0.25,
        'clean_sheets': 1.5,
        'saves': 1.2,
        'penalties_saved': 1.0,
        'ep_next_norm': 1.2,
        'points_per_game_norm': 1.0,
        'selected_by_percent_norm': 0.5,
        'ict_index_norm': 0.3,
        'avg_team_difficulty_norm': 0.5,
        'bps_norm': 0.3,
        'bonus_norm': 0.5,
        'yellow_cards_norm': -0.15,
        'red_cards_norm': -0.15,
        'GW1_total_points': 5,  # New metric for GW1 performance
    },
    "DEF": {
        'total_points': 0.25,
        'clean_sheets': 1.2,
        'goals_scored': 0.5,
        'assists': 0.5,
        'ep_next_norm': 1.2,
        'points_per_game_norm': 1.0,
        'selected_by_percent_norm': 0.5,
        'ict_index_norm': 0.4,
        'avg_team_difficulty_norm': 0.4,
        'bps_norm': 0.3,
        'bonus_norm': 0.4,
        'yellow_cards_norm': -0.15,
        'red_cards_norm': -0.15,
        'defensive_contribution' : 0.5,  # New metric for defensive contribution
        # 'defensive_contribution_per_90': 0.5,  # New metric
        'GW1_total_points': 5,
    },
    "MID": {
        'total_points': 0.25,
        'goals_scored': 1.8,
        'assists': 1.0,
        'penalties_order': 0.5,  # ADDED: penalties_order for midfielders
        'ep_next_norm': 1.2,
        'points_per_game_norm': 1.0,
        'selected_by_percent_norm': 0.5,
        'ict_index_norm': 1.0,
        'avg_team_difficulty_norm': 0.4,
        'bps_norm': 0.3,
        'bonus_norm': 0.4,
        'yellow_cards_norm': -0.3,
        'red_cards_norm': -0.3,
        'GW1_total_points': 5,
    },
    "FWD": {
        'total_points': 0.25,
        'goals_scored': 2.0,
        'assists': 0.8,
        'penalties_order': 0.8,  # ADDED: penalties_order for forwards (higher weight)
        'ep_next_norm': 1.2,
        'points_per_game_norm': 1.0,
        'selected_by_percent_norm': 0.5,
        'ict_index_norm': 1.0,
        'avg_team_difficulty_norm': 0.4,
        'bps_norm': 0.3,
        'bonus_norm': 0.4,
        'yellow_cards_norm': -0.3,
        'red_cards_norm': -0.3,
        'GW1_total_points': 5,
    }
}

# Define position-specific columns for each sheet - UPDATED: includes only relevant normalized columns
POSITION_COLUMNS = {
    "GKP": [
        "first_name", "second_name", "web_name", "team_name", "position",
        "now_cost", "total_points", "minutes", "clean_sheets",
        "saves", "penalties_saved", "status",
        # Normalized columns specific to GKP
        "ep_next_norm", "points_per_game_norm", "selected_by_percent_norm", 
        "ict_index_norm", "bps_norm", "bonus_norm", "yellow_cards_norm", 
        "red_cards_norm", "avg_team_difficulty_norm",
        "expected_score"
    ],
    "DEF": [
        "first_name", "second_name", "web_name", "team_name", "position",
        "now_cost", "total_points", "status",
        # Normalized columns specific to DEF
        "ep_next_norm", "points_per_game_norm", "selected_by_percent_norm", 
        "ict_index_norm", "bps_norm", "bonus_norm", "yellow_cards_norm", 
        "red_cards_norm", "avg_team_difficulty_norm",
        "expected_score"
    ],
    "MID": [
        "first_name", "second_name", "web_name", "team_name", "position",
        "now_cost", "total_points", "penalties_order", "status",
        # Normalized columns specific to MID
        "ep_next_norm", "points_per_game_norm", "selected_by_percent_norm", 
        "ict_index_norm", "bps_norm", "bonus_norm", "yellow_cards_norm", 
        "red_cards_norm", "avg_team_difficulty_norm",
        "expected_score"
    ],
    "FWD": [
        "first_name", "second_name", "web_name", "team_name", "position",
        "now_cost", "total_points", "penalties_order", "status",
        # Normalized columns specific to FWD
        "ep_next_norm", "points_per_game_norm", "selected_by_percent_norm", 
        "ict_index_norm", "bps_norm", "bonus_norm", "yellow_cards_norm", 
        "red_cards_norm", "avg_team_difficulty_norm",
        "expected_score"
    ]
}

# Function to calculate expected score
def calculate_score(row, weight_dict):
    return sum(row.get(col, 0) * weight for col, weight in weight_dict.items())

# Add scores and sort by position
position_map = {
    'GKP': 'Goalkeepers',
    'DEF': 'Defenders',
    'MID': 'Midfielders',
    'FWD': 'Forwards'
}
output = {}

for code, readable in position_map.items():
    subset = players_df[players_df['position'] == code].copy()
    subset['expected_score'] = subset.apply(lambda row: calculate_score(row, WEIGHTS[code]), axis=1)
    subset.sort_values('expected_score', ascending=False, inplace=True)
    
    # Filter to only show position-specific columns
    available_cols = [col for col in POSITION_COLUMNS[code] if col in subset.columns]
    subset = subset[available_cols]
    
    output[readable] = subset

# Save to Excel
with pd.ExcelWriter("OVERALLGW2_fpl_expected_scores_by_position.xlsx") as writer:
    for sheet_name, df in output.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ Excel file 'fpl_expected_scores_by_position.xlsx' created successfully.")
print("\nChanges made:")
print("1. Fixed merge issue: changed left_on from 'team' to 'team_name'")
print("2. Added penalties_order to MID weights (0.5)")
print("3. Added penalties_order to FWD weights (0.8)")
print("4. Each position sheet now shows only relevant features for that position")
print("5. All sheets include the expected_score column")
print("6. Added position-specific normalized columns to each sheet")

# Verify weights sum to 10 for each position
print("\nWeight verification (should sum to 10):")
for pos, weights in WEIGHTS.items():
    total_weight = sum(weights.values())
    print(f"{pos}: {total_weight:.2f}") 

✅ Excel file 'fpl_expected_scores_by_position.xlsx' created successfully.

Changes made:
1. Fixed merge issue: changed left_on from 'team' to 'team_name'
2. Added penalties_order to MID weights (0.5)
3. Added penalties_order to FWD weights (0.8)
4. Each position sheet now shows only relevant features for that position
5. All sheets include the expected_score column
6. Added position-specific normalized columns to each sheet

Weight verification (should sum to 10):
GKP: 12.95
DEF: 11.85
MID: 12.75
FWD: 13.05


In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

# === Load Players & Fixtures Data ===
players_df = pd.read_excel("fpl_players_24_25.xlsx")
fixtures_df = pd.read_excel("fpl_fixtures_named_24_25.xlsx")

# === Filter Only Available Players ===
players_df = players_df[players_df["status"] == "a"]

# === Calculate Average Team Difficulty for First 5 GWs ===
first_5_gws = fixtures_df[fixtures_df['gameweek'] <= 5]

# Home and away team difficulty
home_difficulty = first_5_gws[['gameweek', 'home_team', 'home_team_difficulty']]
home_difficulty.columns = ['gameweek', 'team', 'difficulty']

away_difficulty = first_5_gws[['gameweek', 'away_team', 'away_team_difficulty']]
away_difficulty.columns = ['gameweek', 'team', 'difficulty']

# Combine and average
all_difficulties = pd.concat([home_difficulty, away_difficulty])
avg_difficulty = all_difficulties.groupby('team')['difficulty'].mean().reset_index()
avg_difficulty.columns = ['team_name', 'avg_team_difficulty']

# Merge into players_df
players_df = players_df.merge(avg_difficulty, on='team_name', how='left')

# === Preprocess Penalties Order ===
def encode_penalty_order(x):
    if pd.isna(x): return 0
    if x == 1: return 1.0
    if x == 2: return 0.5
    return 0

players_df['penalty_order_encoded'] = players_df['penalties_order'].apply(encode_penalty_order)

# === Feature Columns ===
features = [
    "now_cost", "minutes", "goals_scored", "assists", "clean_sheets", "goals_conceded",
    "yellow_cards", "red_cards", "penalties_saved", "penalties_missed",
    "saves", "bonus", "bps", "influence", "creativity", "threat", "ict_index",
    "selected_by_percent", "avg_team_difficulty", "penalty_order_encoded", "position"
]

# Drop rows with missing features or targets
players_df = players_df.dropna(subset=features + ['ep_next'])

# === Prepare X and y ===
X = players_df[features].copy()
y = players_df['ep_next'].astype(float)

# One-hot encode position
X = pd.get_dummies(X, columns=['position'], drop_first=True)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Train Model ===
model = RandomForestRegressor(n_estimators=150, random_state=42)
model.fit(X_train, y_train)

# Evaluate
# y_pred = model.predict(X_test)
# Evaluate
y_pred = model.predict(X_test)
rmse = (mean_squared_error(y_test, y_pred)) ** 0.5  # RMSE
print(f"RMSE on test set: {rmse:.3f}")

# rmse = mean_squared_error(y_test, y_pred, squared=False)
# print(f"RMSE on test set: {rmse:.3f}")

# Predict on full dataset
players_df['predicted_ep_next'] = model.predict(X)

# === Export to Excel: One Sheet per Position ===
position_groups = players_df.groupby("position")

with pd.ExcelWriter("fpl_predicted_scores_ml.xlsx") as writer:
    for position, group in position_groups:
        cols_to_save = [
            "first_name", "second_name", "web_name", "team_name", "now_cost", 
            "total_points", "points_per_game", "avg_team_difficulty",
            "selected_by_percent", "ict_index", "bonus", "bps", "penalties_order",
            "ep_next", "predicted_ep_next"
        ]
        group[cols_to_save].sort_values(by="predicted_ep_next", ascending=False).to_excel(
            writer, sheet_name=position, index=False
        )

print("ML predictions saved to: fpl_predicted_scores_ml.xlsx")


RMSE on test set: 0.561
ML predictions saved to: fpl_predicted_scores_ml.xlsx


In [2]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/56.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/56.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/56.8 MB ? eta -:--:--
    --------------------------------------- 0.8/56.8 MB 758.0 kB/s eta 0:01:14
    --------------------------------------- 1.0/56.8 MB 1.1 MB/s eta 0:00:52
    --------------------------------------- 1.3/56.8 MB 1.1 MB/s eta 0:00:53
   - -------------------------------------- 1.6/56.8 MB 1.1 MB/s eta 0:00:52
   - -------------------------------------- 1.8/56.8 MB 1.1 MB/s eta 0:00:51
   - -------------------------------------- 2

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import os

# === Load the dataset ===
players_file = 'fpl_players_24_25.xlsx'
players_df = pd.read_excel(players_file)

# === Preprocess ===
# Filter out unavailable players
players_df = players_df[players_df['status'] == 'a']

# Fill penalty order with 0
players_df['penalties_order'] = players_df['penalties_order'].fillna(0)

fixtures_df = pd.read_excel("fpl_fixtures_named_24_25.xlsx")

#compute difficulty for next 5 gws
team_difficulty = []
for team in players_df['team_name'].unique():
    team_fixtures = fixtures_df[(fixtures_df['home_team'] == team) | (fixtures_df['away_team'] == team)]
    team_fixtures = team_fixtures.sort_values('gameweek').head(5)
    avg_diff = np.mean([
        row['home_team_difficulty'] if row['home_team'] == team else row['away_team_difficulty']
        for _, row in team_fixtures.iterrows()
    ])
    team_difficulty.append((team, avg_diff))
difficulty_dict = dict(team_difficulty)
players_df['avg_difficulty_5gw'] = players_df['team_name'].map(difficulty_dict)

# # Use only GWs 1–5 average difficulty
# players_df['avg_difficulty_5gw'] = players_df[['GW1_difficulty', 'GW2_difficulty', 'GW3_difficulty', 'GW4_difficulty', 'GW5_difficulty']].mean(axis=1)

# === Define features per position ===
position_feature_map = {
    'GKP': [
        'saves', 'penalties_saved', 'clean_sheets', 'goals_conceded',
        'yellow_cards', 'red_cards', 'bps', 'bonus', 'ict_index',
        'ep_next', 'points_per_game', 'selected_by_percent', 'avg_difficulty_5gw'
    ],
    'DEF': [
        'clean_sheets', 'assists', 'goals_scored',
        'bps', 'bonus', 'ict_index', 'yellow_cards', 'red_cards',
        'ep_next', 'points_per_game', 'selected_by_percent', 'avg_difficulty_5gw'
    ],
    'MID': [
        'goals_scored', 'assists', 'expected_goals', 'expected_assists',
        'ict_index', 'bps', 'bonus', 'penalties_order',
        'ep_next', 'points_per_game', 'selected_by_percent', 'avg_difficulty_5gw',
        'yellow_cards', 'red_cards'
    ],
    'FWD': [
        'goals_scored', 'assists', 'expected_goals', 'expected_assists',
        'ict_index', 'bps', 'bonus', 'penalties_order',
        'ep_next', 'points_per_game', 'selected_by_percent', 'avg_difficulty_5gw',
        'yellow_cards', 'red_cards'
    ]
}

# === Results container ===
all_predictions = {}
feature_importances = {}

# === Train separate models per position ===
for position in ['GKP', 'DEF', 'MID', 'FWD']:
    print(f"\nTraining model for: {position}")
    # print(f"players_df: {players_df}")
    df_pos = players_df[players_df['position'] == position].copy()
    # print(f"df_pos: {df_pos}")
    features = position_feature_map[position]
    df_pos = df_pos[features + ['total_points', 'first_name', 'second_name', 'team_name', 'now_cost', 'position']].copy()
    df_pos = df_pos.dropna()

    X = df_pos[features]
    y = df_pos['total_points']
    # print(f"df_pos: {df_pos}")

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    rmse = (mean_squared_error(y_test, y_pred)) ** 0.5
    print(f"{position} RMSE: {rmse:.2f}")

    # Predict for full position data
    df_pos['expected_score'] = model.predict(X)

    # Save predictions
    all_predictions[position] = df_pos[['first_name', 'second_name', 'team_name', 'now_cost', 'position', 'expected_score'] + features].sort_values(by='expected_score', ascending=False)

    # Save feature importances
    feature_imp = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
    feature_importances[position] = feature_imp

# === Export to Excel ===
with pd.ExcelWriter('GW1Based_xgboost_expected_scores_by_position.xlsx', engine='openpyxl') as writer:
    for position, df in all_predictions.items():
        df.to_excel(writer, sheet_name=position, index=False)

print("\n✅ Exported predictions to 'GW1Based_xgboost_expected_scores_by_position.xlsx'")

# === Print Feature Importances Summary ===
print("\n📊 Feature Importances (Top 5 per position):")
for position, imp in feature_importances.items():
    print(f"\n--- {position} ---")
    print(imp.head(5).to_string())



Training model for: GKP
GKP RMSE: 0.47

Training model for: DEF
DEF RMSE: 0.16

Training model for: MID
MID RMSE: 0.00

Training model for: FWD
FWD RMSE: 0.55

✅ Exported predictions to 'GW1Based_xgboost_expected_scores_by_position.xlsx'

📊 Feature Importances (Top 5 per position):

--- GKP ---
clean_sheets       0.928461
saves              0.058898
bps                0.007018
bonus              0.005623
penalties_saved    0.000000

--- DEF ---
points_per_game    0.698151
clean_sheets       0.197335
goals_scored       0.079552
ict_index          0.013474
bps                0.011328

--- MID ---
points_per_game     0.915695
goals_scored        0.084305
expected_goals      0.000000
expected_assists    0.000000
ict_index           0.000000

--- FWD ---
goals_scored        0.663893
ict_index           0.284964
points_per_game     0.050742
expected_goals      0.000402
expected_assists    0.000000


In [31]:
import pandas as pd
import os

# === File paths ===
season_file = r"D:\FPL\fpl_players_24_25.xlsx"
gw1_file = r"D:\FPL\GW1\fpl_players_25_26_GW1Stats.xlsx"
output_file = r"D:\FPL\GW1\fpl_players_merged_with_GW1.xlsx"

# === Load both datasets ===
season_df = pd.read_excel(season_file)
gw1_df = pd.read_excel(gw1_file)

# === Create merge keys ===
season_df['player_key'] = season_df['first_name'].str.strip() + " " + season_df['second_name'].str.strip()
gw1_df['player_key'] = gw1_df['first_name'].str.strip() + " " + gw1_df['second_name'].str.strip()

# === Check for duplicates in keys ===
if season_df['player_key'].duplicated().any():
    print("⚠ Warning: Duplicate player keys in season_df")
if gw1_df['player_key'].duplicated().any():
    print("⚠ Warning: Duplicate player keys in gw1_df")

# === Merge datasets ===
merged_df = pd.merge(
    season_df,
    gw1_df[['player_key', 'total_points']],  # only bring GW1 points
    on="player_key",
    how="left",
    suffixes=("", "_GW1")
)

# === Rename the GW1 column for clarity ===
merged_df.rename(columns={"total_points_GW1": "GW1_total_points"}, inplace=True)

# === Save merged data ===
merged_df.to_excel(output_file, index=False)

print(f"\n✅ Merged file saved to: {output_file}")
print("\n🔎 Sample preview:")
print(merged_df[['player_key', 'total_points', 'GW1_total_points']].head(10))



✅ Merged file saved to: D:\FPL\GW1\fpl_players_merged_with_GW1.xlsx

🔎 Sample preview:
            player_key  total_points  GW1_total_points
0        Mohamed Salah           344               8.0
1         Bryan Mbeumo           236               2.0
2          Cole Palmer           214               3.0
3       Alexander Isak           211               0.0
4           Chris Wood           200              13.0
5         Jarrod Bowen           193               2.0
6        Ollie Watkins           186               2.0
7          Yoane Wissa           185               0.0
8  Luis Díaz Marulanda           183               0.0
9       Erling Haaland           181              13.0


In [32]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import os

# === Load the dataset ===
players_file = r'D:\FPL\GW1\fpl_players_merged_with_GW1.xlsx'
players_df = pd.read_excel(players_file)

# === Preprocess ===
# Filter out unavailable players
players_df = players_df[players_df['status'] == 'a']

# Fill penalty order with 0
players_df['penalties_order'] = players_df['penalties_order'].fillna(0)

fixtures_df = pd.read_excel("fpl_fixtures_named_24_25.xlsx")

#compute difficulty for next 5 gws
team_difficulty = []
for team in players_df['team_name'].unique():
    team_fixtures = fixtures_df[(fixtures_df['home_team'] == team) | (fixtures_df['away_team'] == team)]
    team_fixtures = team_fixtures.sort_values('gameweek').head(5)
    avg_diff = np.mean([
        row['home_team_difficulty'] if row['home_team'] == team else row['away_team_difficulty']
        for _, row in team_fixtures.iterrows()
    ])
    team_difficulty.append((team, avg_diff))
difficulty_dict = dict(team_difficulty)
players_df['avg_difficulty_5gw'] = players_df['team_name'].map(difficulty_dict)

# # Use only GWs 1–5 average difficulty
# players_df['avg_difficulty_5gw'] = players_df[['GW1_difficulty', 'GW2_difficulty', 'GW3_difficulty', 'GW4_difficulty', 'GW5_difficulty']].mean(axis=1)

# === Define features per position ===
position_feature_map = {
    'GKP': [
        'saves', 'penalties_saved', 'clean_sheets', 'goals_conceded',
        'yellow_cards', 'red_cards', 'bps', 'bonus', 'ict_index',
        'ep_next', 'points_per_game', 'selected_by_percent', 'avg_difficulty_5gw', 'GW1_total_points'
    ],
    'DEF': [
        'clean_sheets', 'assists', 'goals_scored',
        'bps', 'bonus', 'ict_index', 'yellow_cards', 'red_cards',
        'ep_next', 'points_per_game', 'selected_by_percent', 'avg_difficulty_5gw', 'GW1_total_points'
    ],
    'MID': [
        'goals_scored', 'assists', 'expected_goals', 'expected_assists',
        'ict_index', 'bps', 'bonus', 'penalties_order',
        'ep_next', 'points_per_game', 'selected_by_percent', 'avg_difficulty_5gw',
        'yellow_cards', 'red_cards', 'GW1_total_points'
    ],
    'FWD': [
        'goals_scored', 'assists', 'expected_goals', 'expected_assists',
        'ict_index', 'bps', 'bonus', 'penalties_order',
        'ep_next', 'points_per_game', 'selected_by_percent', 'avg_difficulty_5gw',
        'yellow_cards', 'red_cards', 'GW1_total_points'
    ]
}

# === Results container ===
all_predictions = {}
feature_importances = {}

# === Train separate models per position ===
for position in ['GKP', 'DEF', 'MID', 'FWD']:
    print(f"\nTraining model for: {position}")
    # print(f"players_df: {players_df}")
    df_pos = players_df[players_df['position'] == position].copy()
    # print(f"df_pos: {df_pos}")
    features = position_feature_map[position]
    df_pos = df_pos[features + ['total_points', 'first_name', 'second_name', 'team_name', 'now_cost', 'position']].copy()
    df_pos = df_pos.dropna()

    X = df_pos[features]
    y = df_pos['total_points']
    # print(f"df_pos: {df_pos}")

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    rmse = (mean_squared_error(y_test, y_pred)) ** 0.5
    print(f"{position} RMSE: {rmse:.2f}")

    # Predict for full position data
    df_pos['expected_score'] = model.predict(X)

    # Save predictions
    all_predictions[position] = df_pos[['first_name', 'second_name', 'team_name', 'now_cost', 'position', 'expected_score'] + features].sort_values(by='expected_score', ascending=False)

    # Save feature importances
    feature_imp = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
    feature_importances[position] = feature_imp

# === Export to Excel ===
with pd.ExcelWriter('Overall_xgboost_expected_scores_by_position.xlsx', engine='openpyxl') as writer:
    for position, df in all_predictions.items():
        df.to_excel(writer, sheet_name=position, index=False)

print("\n✅ Exported predictions to 'Overall_xgboost_expected_scores_by_position.xlsx'")

# === Print Feature Importances Summary ===
print("\n📊 Feature Importances (Top 5 per position):")
for position, imp in feature_importances.items():
    print(f"\n--- {position} ---")
    print(imp.head(5).to_string())



Training model for: GKP
GKP RMSE: 9.24

Training model for: DEF
DEF RMSE: 7.39

Training model for: MID
MID RMSE: 6.99

Training model for: FWD
FWD RMSE: 14.57

✅ Exported predictions to 'Overall_xgboost_expected_scores_by_position.xlsx'

📊 Feature Importances (Top 5 per position):

--- GKP ---
saves              8.419724e-01
clean_sheets       1.238157e-01
bps                3.402555e-02
ep_next            1.862528e-04
points_per_game    1.817751e-08

--- DEF ---
bps                    0.691104
clean_sheets           0.245673
points_per_game        0.022104
selected_by_percent    0.018233
ict_index              0.015662

--- MID ---
ict_index         0.825816
bps               0.093249
expected_goals    0.030646
bonus             0.018892
goals_scored      0.014438

--- FWD ---
goals_scored          0.919916
ict_index             0.037361
expected_goals        0.036324
yellow_cards          0.002088
avg_difficulty_5gw    0.001369


In [5]:
pip install xlsxwriter

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [33]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, LpBinary, LpStatus, value

# ---------- LOAD DATA ----------
excel_path = r"D:\FPL\Overall_xgboost_expected_scores_by_position.xlsx"  # Change to your Excel file
positions = ['GKP','DEF','MID','FWD']
all_players = []

for pos in positions:
    df = pd.read_excel(excel_path, sheet_name=pos)
    df['position'] = pos.upper()
    all_players.append(df)

df_all = pd.concat(all_players, ignore_index=True)

# Required columns
df_all = df_all[['first_name', 'second_name', 'team_name', 'expected_score', 'position', 'now_cost']].dropna()
df_all['name'] = df_all['first_name'] + ' ' + df_all['second_name']
df_all['cost'] = df_all['now_cost'] / 10

# ---------- ILP MODEL ----------
model = LpProblem("FPL_15_Man_Squad_With_Captain", LpMaximize)

# Decision variables
player_vars = {i: LpVariable(f"select_{row['name']}_{i}", cat=LpBinary) for i, row in df_all.iterrows()}
captain_vars = {i: LpVariable(f"captain_{row['name']}_{i}", cat=LpBinary) for i, row in df_all.iterrows()}

# ---------- OBJECTIVE FUNCTION ----------
model += lpSum([
    (2 * row['expected_score'] * captain_vars[i]) +
    (row['expected_score'] * (player_vars[i] - captain_vars[i]))
    for i, row in df_all.iterrows()
])

# ---------- CONSTRAINTS ----------

# Squad size
model += lpSum(player_vars.values()) == 15

# Position limits
model += lpSum([player_vars[i] for i in df_all[df_all['position'] == 'GKP'].index]) == 2
model += lpSum([player_vars[i] for i in df_all[df_all['position'] == 'DEF'].index]) == 5
model += lpSum([player_vars[i] for i in df_all[df_all['position'] == 'MID'].index]) == 5
model += lpSum([player_vars[i] for i in df_all[df_all['position'] == 'FWD'].index]) == 3

# Max 3 per team
for team in df_all['team_name'].unique():
    model += lpSum([player_vars[i] for i in df_all[df_all['team_name'] == team].index]) <= 3

# Budget
model += lpSum([row['cost'] * player_vars[i] for i, row in df_all.iterrows()]) <= 100.0

# Exactly 1 captain
model += lpSum(captain_vars.values()) == 1

# A captain must be one of the selected players
for i in df_all.index:
    model += captain_vars[i] <= player_vars[i]

# ---------- SOLVE ----------
model.solve()

print(f"Solver Status: {LpStatus[model.status]}\n")

# ---------- RESULTS ----------
selected = []
for i, row in df_all.iterrows():
    if player_vars[i].value() == 1:
        selected.append({
            'Name': row['name'],
            'Team': row['team_name'],
            'Position': row['position'],
            'Cost': row['cost'],
            'Expected_Score': row['expected_score'],
            'Captain': 'Yes' if captain_vars[i].value() == 1 else '',
            'Starting_XI': '',  # placeholder for now
            'Points': ''        # placeholder for later
        })

squad_df = pd.DataFrame(selected).sort_values(by='Position').reset_index(drop=True)

# Total score and cost
total_score = squad_df['Expected_Score'].sum() + squad_df[squad_df['Captain'] == 'Yes']['Expected_Score'].sum()
total_cost = squad_df['Cost'].sum()

# ---------- STARTING XI & BENCH ----------
# Pick best XI based on highest expected score but ensure 1 GK in XI
gk_df = squad_df[squad_df['Position'] == 'GKP'].sort_values(by='Expected_Score', ascending=False)
outfield_df = squad_df[squad_df['Position'] != 'GKP'].sort_values(by='Expected_Score', ascending=False)

starting_gk = gk_df.head(1)       # best GK
bench_gk = gk_df.tail(1)          # backup GK
starting_outfield = outfield_df.head(10)  # best 10 outfielders
bench_outfield = outfield_df.tail(3)      # remaining 3

starting_xi = pd.concat([starting_gk, starting_outfield])
bench = pd.concat([bench_gk, bench_outfield])

# Mark in squad_df
squad_df.loc[squad_df['Name'].isin(starting_xi['Name']), 'Starting_XI'] = 'Yes'
squad_df.loc[squad_df['Name'].isin(bench['Name']), 'Starting_XI'] = 'Bench'

# Formation
formation_counts = starting_xi['Position'].value_counts()
formation = f"{formation_counts.get('DEF',0)}-{formation_counts.get('MID',0)}-{formation_counts.get('FWD',0)}"

# ---------- PRINT ----------
print(squad_df.to_string(index=False))
print(f"\nTotal Cost: £{total_cost:.1f}M")
print(f"Total Expected Score (w/ Captain Bonus): {total_score:.2f}")
print(f"Ideal Formation (Starting XI): {formation}")

# ---------- SAVE ----------
output_path = "GW2_Overall_XGBOOST_best_15_squad_with_starting_XI.xlsx"
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    squad_df.to_excel(writer, index=False, sheet_name="Squad")
    starting_xi.to_excel(writer, index=False, sheet_name="Starting_XI")
    bench.to_excel(writer, index=False, sheet_name="Bench")
    pd.DataFrame([{"Ideal_Formation": formation}]).to_excel(writer, index=False, sheet_name="Formation")

print(f"\n✅ Squad + Starting XI + Bench saved to: {output_path}")


Solver Status: Optimal

                  Name           Team Position  Cost  Expected_Score Captain Starting_XI Points
        Joško Gvardiol       Man City      DEF   6.0      152.481628                 Yes       
     Nikola Milenković  Nott'm Forest      DEF   5.5      144.879913                 Yes       
    Daniel Muñoz Mejía Crystal Palace      DEF   5.5      141.819702                 Yes       
            Marc Guéhi Crystal Palace      DEF   4.5      130.231659               Bench       
     Aaron Wan-Bissaka       West Ham      DEF   4.5      118.167564               Bench       
            Chris Wood  Nott'm Forest      FWD   7.5      199.361832                 Yes       
          Jarrod Bowen       West Ham      FWD   8.0      192.825134                 Yes       
Raúl Jiménez Rodríguez         Fulham      FWD   6.5      146.945770                 Yes       
             Matz Sels  Nott'm Forest      GKP   5.0      149.440781                 Yes       
        Dean Hen

In [38]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, LpBinary, LpStatus

# ---------- LOAD DATA ----------
excel_path = r"D:\FPL\OVERALLGW2_fpl_expected_scores_by_position.xlsx"
positions_map = {
    'Goalkeepers': 'GKP',
    'Defenders': 'DEF',
    'Midfielders': 'MID',
    'Forwards': 'FWD'
}

all_players = []
for sheet, pos in positions_map.items():
    df = pd.read_excel(excel_path, sheet_name=sheet)
    df['position'] = pos
    all_players.append(df)

df_all = pd.concat(all_players, ignore_index=True)

# Required columns
df_all = df_all[['first_name', 'second_name', 'team_name', 'expected_score', 'position', 'now_cost']].dropna()
df_all['name'] = df_all['first_name'] + ' ' + df_all['second_name']
df_all['cost'] = df_all['now_cost'] / 10

# ---------- ILP MODEL ----------
model = LpProblem("FPL_15_Man_Squad_With_XI_And_Captain", LpMaximize)

# Decision variables
player_vars = {i: LpVariable(f"select_{row['name']}_{i}", cat=LpBinary) for i, row in df_all.iterrows()}
xi_vars = {i: LpVariable(f"xi_{row['name']}_{i}", cat=LpBinary) for i, row in df_all.iterrows()}
captain_vars = {i: LpVariable(f"captain_{row['name']}_{i}", cat=LpBinary) for i, row in df_all.iterrows()}

# ---------- OBJECTIVE FUNCTION ----------
model += lpSum([
    (2 * row['expected_score'] * captain_vars[i]) +
    (row['expected_score'] * (xi_vars[i] - captain_vars[i]))  # only XI scores count
    for i, row in df_all.iterrows()
])

# ---------- CONSTRAINTS ----------

# Squad size
model += lpSum(player_vars.values()) == 15

# Position limits
model += lpSum([player_vars[i] for i in df_all[df_all['position'] == 'GKP'].index]) == 2
model += lpSum([player_vars[i] for i in df_all[df_all['position'] == 'DEF'].index]) == 5
model += lpSum([player_vars[i] for i in df_all[df_all['position'] == 'MID'].index]) == 5
model += lpSum([player_vars[i] for i in df_all[df_all['position'] == 'FWD'].index]) == 3

# Max 3 per team
for team in df_all['team_name'].unique():
    model += lpSum([player_vars[i] for i in df_all[df_all['team_name'] == team].index]) <= 3

# Budget
model += lpSum([row['cost'] * player_vars[i] for i, row in df_all.iterrows()]) <= 100.0

# Starting XI size
model += lpSum(xi_vars.values()) == 11

# XI must be subset of squad
for i in df_all.index:
    model += xi_vars[i] <= player_vars[i]

# Bench must include 1 GKP
model += lpSum([xi_vars[i] for i in df_all[df_all['position'] == 'GKP'].index]) == 1

# Valid formation constraints
model += lpSum([xi_vars[i] for i in df_all[df_all['position'] == 'DEF'].index]) >= 3
model += lpSum([xi_vars[i] for i in df_all[df_all['position'] == 'MID'].index]) >= 3
model += lpSum([xi_vars[i] for i in df_all[df_all['position'] == 'FWD'].index]) >= 1

# Exactly 1 captain
model += lpSum(captain_vars.values()) == 1

# Captain must be in XI
for i in df_all.index:
    model += captain_vars[i] <= xi_vars[i]

# ---------- SOLVE ----------
model.solve()

# ---------- RESULTS ----------
status = LpStatus[model.status]
print(f"Solver Status: {status}\n")

if status != "Optimal":
    print("⚠️ Optimization was not successful. Check constraints or budget.")
else:
    # Get selected squad
    selected = []
    for i, row in df_all.iterrows():
        if player_vars[i].value() == 1:
            starter = 'Yes' if xi_vars[i].value() == 1 else 'Bench'
            captain = 'Yes' if captain_vars[i].value() == 1 else ''
            
            # Points calculation
            if starter == 'Bench':
                points = 0
            elif captain == 'Yes':
                points = 2 * row['expected_score']
            else:
                points = row['expected_score']
            
            selected.append({
                'Name': row['name'],
                'Team': row['team_name'],
                'Position': row['position'],
                'Cost': row['cost'],
                'Expected_Score': row['expected_score'],
                'Starter': starter,
                'Captain': captain,
                'Points': ''
            })

    squad_df = pd.DataFrame(selected).sort_values(by=['Starter','Position'], ascending=[False, True])
    total_score = squad_df['Expected_Score'].sum()
    total_cost = squad_df['Cost'].sum()

    # Formation (DEF-MID-FWD counts among XI)
    def_count = (squad_df[(squad_df['Starter'] == 'Yes') & (squad_df['Position'] == 'DEF')].shape[0])
    mid_count = (squad_df[(squad_df['Starter'] == 'Yes') & (squad_df['Position'] == 'MID')].shape[0])
    fwd_count = (squad_df[(squad_df['Starter'] == 'Yes') & (squad_df['Position'] == 'FWD')].shape[0])
    formation = f"{def_count}-{mid_count}-{fwd_count}"

    # Display squad
    print(squad_df.to_string(index=False))
    print(f"\nFormation: {formation}")
    print(f"Total Cost: £{total_cost:.1f}M")
    print(f"Total Expected Score (XI + Captain Bonus): {total_score:.2f}")

    # ---------- SAVE TO EXCEL ----------
    output_path = "GW2_Overall_best_15_squad_with_xi_and_captain_weights.xlsx"
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        squad_df.to_excel(writer, index=False, sheet_name="Squad")

        # Add metadata sheet
        meta = pd.DataFrame({
            'Metric': ['Formation', 'Total Cost', 'Total Expected XI Score', 'Solver Status'],
            'Value': [formation, f"£{total_cost:.1f}M", f"{total_score:.2f}", status]
        })
        meta.to_excel(writer, index=False, sheet_name="Summary")
    
    print(f"\n✅ Squad with XI, Captain & Formation saved to: {output_path}")


Solver Status: Optimal

                    Name           Team Position  Cost  Expected_Score Starter Captain Points
         Maxence Lacroix Crystal Palace      DEF   5.0      257.939255     Yes     Yes       
Murillo Costa dos Santos  Nott'm Forest      DEF   5.5      247.526494     Yes               
         James Tarkowski        Everton      DEF   5.5      239.041133     Yes               
                Dan Burn      Newcastle      DEF   5.0      234.794980     Yes               
          Nathan Collins      Brentford      DEF   5.0      222.748709     Yes               
          Erling Haaland       Man City      FWD  14.0      160.552319     Yes               
              Chris Wood  Nott'm Forest      FWD   7.5      160.474226     Yes               
         Jordan Pickford        Everton      GKP   5.5      218.127438     Yes               
           Mohamed Salah      Liverpool      MID  14.5      201.189618     Yes               
         Antoine Semenyo    Bournemo